# **APIs**

API stands for Application Program Interface and is a way for two or more computer programmes to communicate with each other.

REST (Representational State Transfer) APIs allow communication between a client (your program) and a server (usually online) using HTTP (HyperText Transfer Protocol) to communicate.

A HTTP request is sent from client to server containing the instructions in a file (usually .json). The server then executes the instructions and sends a HTTP response which contains the result.

Let's look at an example.

In [ ]:
import requests
import json
import pandas

We first define the HTTP request providing the appropriate URL.

In [ ]:
country = 'Italy'
daterange = '20211215-20220115'
url = ( 'https://covidmap.umd.edu/api/resources?indicator=covid&type=smoothed&country=' +
    country + '&daterange=' + daterange )

We then send request via API and collect the text response using the `get()` function.

In [ ]:
response = requests.get(url)
print(response)

We can get a number of API responses the key ones to remember are:

*   200 - success
*   400 - bad request (i.e., the error is in our code)
*   500 - internal server error (i.e., the error is on the server side)


We then transform the text response in a json file. a json is a JavaScript Object Notation consisting of name-object pairs and punctuation in the form of brackets, parentheses, semicolons and colons (basically a big dictionary!)

In [ ]:
jsonData = json.loads(response.text)
print(jsonData)

And lastly convert the json to a pandas dataframe - ready for us to use.

In [ ]:
df = pandas.DataFrame.from_dict(jsonData['data'])
df.head()

## **Using World Bank Data API**

You can find the documentation on the API interface and practical examples [here](https://wbdata.readthedocs.io/en/stable/) and [here](https://blogs.worldbank.org/opendata/accessing-world-bank-data-apis-python-r-ruby-stata) respectively.

Let's replicate the example code below.

We first need to install the `wbdata` library which contains the API call.

In [ ]:
!pip3 install -U wbdata

We then import the libraries.

In [ ]:
import wbdata
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

The `get_country()` function gives us a list of all countries.  

In [ ]:
wbdata.get_countries()

We can pick the countries that interest us.

In [ ]:
countries = ["USA","GBR","ITA"]

The `get_indicator()` function gives us a list of all indicators.

In [ ]:
wbdata.get_indicators()

We can choose the indicator that we want.

In [ ]:
indicators = {'NY.GNP.PCAP.CD':'GNI per Capita'}

We then make the call using the country and indicators parameters.

In [ ]:
df = wbdata.get_dataframe(indicators, country=countries, parse_dates=False)

The df is given as a pivot table. Pandas' `.unstack()` method helps reshape it into something plottable.

In [ ]:
dfu = df.unstack(level=0)
dfu

We can now plot the GNI for the selected countries.

In [ ]:
dfu.plot();
plt.legend(loc='best');
plt.title("GNI Per Capita ($USD, Atlas Method)");
plt.xlabel('Date');
plt.ylabel('GNI Per Capita ($USD, Atlas Method');

We can also plot two indicators together if we want to compare their trends.
Let us see the relationship between GDP and population un China.

We first select the country and indicators of interest.

In [ ]:
chn = ['CHN']
gdp_indicator = {'NY.GDP.PCAP.CD':'GDP'}
pop_indicator = {'SP.POP.TOTL':'POP'}

Then get the data by calling the World Bank Data API.

In [ ]:
gdp = wbdata.get_dataframe(gdp_indicator, country=chn, parse_dates=False)
pop = wbdata.get_dataframe(pop_indicator , country=chn, parse_dates=False)

To make the two indicators comparable in out plot we standardise them.

In [ ]:
gdp = (gdp-gdp.mean())/gdp.std()
pop = (pop-pop.mean())/pop.std()

And then make sure that years are in the right order.

In [ ]:
gdp = gdp.sort_index(ascending=True)
pop = pop.sort_index(ascending=True)

We then plot the two indicators.

In [ ]:
fig, ax = plt.subplots(dpi=200)
plt.rcParams.update({'font.size': 10})
fig.subplots_adjust(wspace=0)
ax.plot(gdp, linewidth=2.0, label='GDP')
ax.plot(pop, linewidth=2.0, label='POP')
plt.xticks(rotation='vertical')
plt.xticks(np.arange(0, 61, 5))
plt.title('GDP vs. POP in China');
plt.xlabel('year');
plt.ylabel('standardised value');
plt.legend()

# **Exercise**

Download the first 10000 violent incidents acoross the world reported in 2020 by the Uppsala Conflict Data Program and create plots as indicated, following examples in the past tutorials and examples contained in the plotting_example.ipynb file.

1. Familiarize youself with the API documentation, formulate a query to download the data and return the response as a dataframe following the example above.

2. After checking that you have downloaded the data correctly and what information is available, create a barchart reporting violent incidents by country.

3. Create a second barchart like the one above but using colors to differentiate types of violence. Make sure a legend and labels are present.

4. Side by side, create three histograms reporting the distribution of violent incidents by type of violence in the sample population of countries.